## Exemples de robots

En aques exemple veurem diferents tipus de sensors i robots que podem utilitzar en `aitk.robots` i com podem interactuar amb ells.

In [ ]:
!pip install aitk aitk.robots ipykernel

: 

In [2]:
from aitk.robots import (
    World, Scribbler, 
    RangeSensor, LightSensor, SmellSensor, 
    Camera
)
from random import random

### Un món amb parets, una bombeta i menjar


Hi ha una font de menjar situada prop de l'origen del món darrere d'una petita paret blava inclinada. Hi ha una font de llum prop del centre del món amagada a la cantonada de les parets cian i taronja.

Utilitzem menjar i llum per atraure els robots cap a ells. Els robots poden veure la llum i el menjar i poden moure's cap a ells. Els robots també poden veure les parets i evitar-les. Òbviament, els robots no menjen, aquest element, però, ens permetrà veure com els robots poden interactuar amb els objectes del món.

In [ ]:
# Definim el món
# Creem un món de 200x150 amb una escala de 5.0

world = World(width=200, height=150, scale=5.0)
world.add_wall("cyan", 80, 50, 90, 150)
world.add_wall("orange", 90, 50, 110, 60)
world.add_wall("blue", 0, 35, 25, 30, box=False)
world.add_bulb("yellow", 100, 70, 0, 75.0)
world.add_food(15, 10, 50)

# Mostrem el món i demanem que s'actualitzi automàticament
world.watch(width="700px")

### El robot vermell utilitza sensors IR

El robot vermell utilitza sensors IR per detectar i evitar obstacles. Pot determinar cap a quin costat girar basant-se en si l'obstacle està més a prop del seu front esquerra o front dret.

In [4]:
robot1 = Scribbler(x=150, y=100, a=35, color="red", name="red")
robot1.add_device(RangeSensor(position=(6,-6),width=57.3,max=20,name="left-ir"))
robot1.add_device(RangeSensor(position=(6,6),width=57.3,max=20,name="right-ir"))

world.add_robot(robot1)

### El robot rosa utilitza sensors d'olor i una càmera

El robot rosa utilitza sensors d'olor per trobar menjar. També té una càmera que pot veure les característiques del món. Pot girar cap al menjar detectat a la seva esquerra o dreta.

In [ ]:
robot2 = Scribbler(x=40, y=130, a=75, color="pink", name="pink")
robot2.state["timer"] = 0 # utilitzem el diccionari state per guardar variables.
                          # Aquesta variable ens servirà per controlar el temps
robot2.add_device(Camera())
robot2.add_device(SmellSensor(position=(6,-6),name="left-smell"))
robot2.add_device(SmellSensor(position=(6,6),name="right-smell"))

world.add_robot(robot2)

: 

### El robot groc utilitza sensors de llum

El robot groc utilitza sensors de llum per trobar una font de llum. Pot girar cap a la llum detectada a la seva esquerra o dreta.

In [6]:
robot3 = Scribbler(x=60, y=30, a=0, color="yellow", name="yellow")
robot3.add_device(LightSensor(position=(6,-6), name="left-light"))
robot3.add_device(LightSensor(position=(6,6), name="right-light"))

world.add_robot(robot3)

In [ ]:
world.watch(width="700px")

In [ ]:
robot2['camera'].watch()

### El controlador del robot vermell deambula

El robot vermell deambula pel món evitant els obstacles que troba. Sempre es mou cap endavant, però tria quantitats de rotació aleatòries quan els seus sensors de gamma frontal estan nets.

In [9]:
def deambula_amb_infrarojos(robot):
    max_val = robot["left-ir"].get_max()
    if robot["left-ir"].get_distance() < max_val:
        robot.move(0.1, -0.3)
    elif robot["right-ir"].get_distance() < max_val:
        robot.move(0.1, 0.3)
    else:
        robot.move(1, random()-0.5)

### El controlador del robot rosa deambula

El robot rosa deambula pel món cercant menjar i evitant obstacles retrocedint quan es queda aturat. Utilitza una variable d'estat anomenada *timer* per assegurar-se que fins i tot en presència d'un olor sempre fa almenys cinc passos enrere quan troba un obstacle. Si s'apropa prou al menjar, intenta menjar-lo, i si ho aconsegueix acaba l'execució retornant True. De tant en tant tria un nou moviment erràtic aleatori. Si cap dels seus casos de prova s'activa, repeteix el moviment anterior.

In [10]:
def busca_menjar(robot):
    left_smell = robot["left-smell"].get_reading()
    right_smell = robot["right-smell"].get_reading()
    diff = left_smell - right_smell
    if left_smell+right_smell>1.95 and robot.eat():
        robot.move(0,0)
        robot.speak("he menjat!")
        return True
    # Utilitzem un temporitzador per assegurar que torne enrere
    # durant un temps mínim de 5 iteracions
    if robot.state["timer"] > 5:
        robot.state["timer"] = 0
    if robot.stalled:
        robot.state["timer"] = 1
        robot.reverse()
    elif robot.state["timer"] > 0:
        robot.state["timer"] += 1
    elif diff > 0.03:
        robot.move(1, 0.3)
    elif diff < -0.03:
        robot.move(1, -0.3)
    elif world.time%2 == 0:
        robot.move(1.0, random()*0.5-0.25)
    else:
        pass #continua en la mateixa direcció

### El controlador del robot groc cerca la llum

L'objectiu del robot groc és trobar i acostar-se a una font de llum. Quan no detecta llum, el robot deambula. Quan detecta més llum a un costat, gira cap a la llum. Si el robot es queda aturat, inverteix el seu moviment. De tant en tant genera un nou moviment erràtic aleatori. Quan cap dels tests s'activa, el robot continuarà fent el moviment anterior. Quan les seves lectures de llum superen un llindar, s'atura i informa que ha trobat la llum. Retornant `True` acaba l'execució.

In [16]:
def search_light(robot):
    left_light = robot["left-light"].get_brightness()
    right_light = robot["right-light"].get_brightness()
    diff = left_light - right_light
    if left_light + right_light > 1.9:
        robot.move(0,0)
        robot.speak("he trobat la llum!")
        return True
    if robot.stalled:
        robot.reverse()
    elif diff > 0.01:
        robot.move(1,0.5)
    elif diff < -0.01:
        robot.move(1,-0.5)   
    elif world.time%2 == 0:
        robot.move(1.0, random()*0.5-0.25)
    else:
        pass #continua en la mateixa direcció

In [ ]:
world.reset()
robot2.state["timer"] = 0 #resseteja el temporitzador
world.set_seed(random())
world.seconds(40,[deambula_amb_infrarojos, busca_menjar, search_light], real_time=True)

Podem utilitzar `real_time=False` per executar el programa més ràpidament, però sense esperar els temps de pausa que hem definit.

Áquesta funció ens serà molt útil per entrenar models i fer simulacions més ràpides.

In [ ]:
world.reset()
robot2.state["timer"] = 0 #resseteja el temporitzador
world.set_seed(random())
world.seconds(40,[deambula_amb_infrarojos, busca_menjar, search_light], real_time=False)